# 准备数据

In [23]:
import random

N = 2000

# 1. 随机数据
randomData = list(range(N))
random.shuffle(randomData)

# 2. 倒序数据
reversedData = list(range(N-1,-1,-1))

# 3. 几乎已经排序好的数据
almostSortedData = list(range(N))
almostSortedData[1],almostSortedData[-2] = almostSortedData[-2], almostSortedData[1]

# 4. 已经排序好的数据
sortedData = list(range(N))

# 5. 全部重复的数据
repeatData = [1] * N

# 各种排序算法的代码实现

In [2]:
# 所有排序算法要调用的函数
def swap(data, i, j) -> None:
    data[i], data[j] = data[j], data[i]

## 选择排序

In [3]:
def select_sort(data, reverse=False) -> bool:
    '''选择排序
    参数：
        data：待排序的列表
        reverse: 是否倒序排序，默认False
    返回值：
        bool类型：是否排序成功
    '''
    if not all([str(i).isdigit() for i in data]):
        return False
    
    lenData = len(data)
    
    for i in range(lenData):
        minInd = i
        for j in range(i+1, lenData):
            if data[j] < data[minInd]:
                minInd = j 
        swap(data, i, minInd)
    
    if reverse:
        data = data[::-1]
    return True

In [4]:
import copy

data = copy.copy(randomData)
res = select_sort(data)
res, data, randomData

(True,
 [0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31,
  32,
  33,
  34,
  35,
  36,
  37,
  38,
  39,
  40,
  41,
  42,
  43,
  44,
  45,
  46,
  47,
  48,
  49,
  50,
  51,
  52,
  53,
  54,
  55,
  56,
  57,
  58,
  59,
  60,
  61,
  62,
  63,
  64,
  65,
  66,
  67,
  68,
  69,
  70,
  71,
  72,
  73,
  74,
  75,
  76,
  77,
  78,
  79,
  80,
  81,
  82,
  83,
  84,
  85,
  86,
  87,
  88,
  89,
  90,
  91,
  92,
  93,
  94,
  95,
  96,
  97,
  98,
  99,
  100,
  101,
  102,
  103,
  104,
  105,
  106,
  107,
  108,
  109,
  110,
  111,
  112,
  113,
  114,
  115,
  116,
  117,
  118,
  119,
  120,
  121,
  122,
  123,
  124,
  125,
  126,
  127,
  128,
  129,
  130,
  131,
  132,
  133,
  134,
  135,
  136,
  137,
  138,
  139,
  140,
  141,
  142,
  143,
  144,
  145,
  146,
  147,
  148,
  149,
  150,
  151,
  152,
  153,
  154,
  155,
  156,
  15

## 插入排序

In [5]:
def insert_sort(data, reverse=False) -> bool:
    '''插入排序
    参数：
        data：待排序的列表
        reverse: 是否倒序排序，默认False
    返回值：
        bool类型：是否排序成功
    '''
    if not all([str(i).isdigit() for i in data]):
        return False
    
    lenData = len(data)
    
    for i in range(1, lenData):
        for j in range(i, 0, -1):
            if data[j] < data[j-1]:
                swap(data, j, j-1)
    
    if reverse:
        data = data[::-1]
    return True

In [6]:
import copy

data = copy.copy(randomData)
res = insert_sort(data)
res, data, randomData

(True,
 [0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31,
  32,
  33,
  34,
  35,
  36,
  37,
  38,
  39,
  40,
  41,
  42,
  43,
  44,
  45,
  46,
  47,
  48,
  49,
  50,
  51,
  52,
  53,
  54,
  55,
  56,
  57,
  58,
  59,
  60,
  61,
  62,
  63,
  64,
  65,
  66,
  67,
  68,
  69,
  70,
  71,
  72,
  73,
  74,
  75,
  76,
  77,
  78,
  79,
  80,
  81,
  82,
  83,
  84,
  85,
  86,
  87,
  88,
  89,
  90,
  91,
  92,
  93,
  94,
  95,
  96,
  97,
  98,
  99,
  100,
  101,
  102,
  103,
  104,
  105,
  106,
  107,
  108,
  109,
  110,
  111,
  112,
  113,
  114,
  115,
  116,
  117,
  118,
  119,
  120,
  121,
  122,
  123,
  124,
  125,
  126,
  127,
  128,
  129,
  130,
  131,
  132,
  133,
  134,
  135,
  136,
  137,
  138,
  139,
  140,
  141,
  142,
  143,
  144,
  145,
  146,
  147,
  148,
  149,
  150,
  151,
  152,
  153,
  154,
  155,
  156,
  15

## 冒泡排序

In [7]:
def bubble_sort(data, reverse=False) -> bool:
    '''冒泡排序
    参数：
        data：待排序的列表
        reverse: 是否倒序排序，默认False
    返回值：
        bool类型：是否排序成功
    '''
    if not all([str(i).isdigit() for i in data]):
        return False
    
    lenData = len(data)
    
    for i in range(lenData-1):
        for j in range(lenData-1-i):
            if data[j] > data[j+1]:
                swap(data, j, j+1)
    
    if reverse:
        data = data[::-1]
    return True

In [8]:
import copy

data = copy.copy(reversedData)
res = bubble_sort(data)
res, data, randomData

(True,
 [0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31,
  32,
  33,
  34,
  35,
  36,
  37,
  38,
  39,
  40,
  41,
  42,
  43,
  44,
  45,
  46,
  47,
  48,
  49,
  50,
  51,
  52,
  53,
  54,
  55,
  56,
  57,
  58,
  59,
  60,
  61,
  62,
  63,
  64,
  65,
  66,
  67,
  68,
  69,
  70,
  71,
  72,
  73,
  74,
  75,
  76,
  77,
  78,
  79,
  80,
  81,
  82,
  83,
  84,
  85,
  86,
  87,
  88,
  89,
  90,
  91,
  92,
  93,
  94,
  95,
  96,
  97,
  98,
  99,
  100,
  101,
  102,
  103,
  104,
  105,
  106,
  107,
  108,
  109,
  110,
  111,
  112,
  113,
  114,
  115,
  116,
  117,
  118,
  119,
  120,
  121,
  122,
  123,
  124,
  125,
  126,
  127,
  128,
  129,
  130,
  131,
  132,
  133,
  134,
  135,
  136,
  137,
  138,
  139,
  140,
  141,
  142,
  143,
  144,
  145,
  146,
  147,
  148,
  149,
  150,
  151,
  152,
  153,
  154,
  155,
  156,
  15

## 快排
**基本原理**（分治法思想）  
1. 选定Pivot中心轴
2. 遍历尚未处理的数据（乱堆），将小于Pivot的数字放在Pivot左边（小堆），大于Pivot的数字放在Pivot右边（大堆）
3. 分别对左右子序列重复前两步操作

**使用装饰器实现尾递归优化** （避免数据过大时栈溢出）  
优化后仍然会栈溢出，尚不清楚原因

In [16]:
import sys

class TailRecurseException(Exception):
    def __init__(self, args, kwargs):
        self.args = args
        self.kwargs = kwargs

def tail_call_optimized(g):
    """
    This function decorates a function with tail call
    optimization. It does this by throwing an exception
    if it is it's own grandparent, and catching such
    exceptions to fake the tail call optimization.

    This function fails if the decorated
    function recurses in a non-tail context.
    """
    def func(*args, **kwargs):
        f = sys._getframe()
        if f.f_back and f.f_back.f_back \
            and f.f_back.f_back.f_code == f.f_code:
            # 抛出异常
            raise TailRecurseException(args, kwargs)
        else:
            while 1:
                try:
                    return g(*args, **kwargs)
                except TailRecurseException as e:
                    args = e.args
                    kwargs = e.kwargs
    func.__doc__ = g.__doc__
    func.__name__ = g.__name__
    return func


**快排的代码实现以及3种优化方式**

In [25]:
import random
# @tail_call_optimized
def quick_sort(data, reverse=False, optimize=0b111) -> bool:
    '''快速排序
    参数：
        data：待排序的列表
        reverse: 是否倒序排序，默认False
        optimize: 采取哪些优化方案，默认采取所有优化方案
            * 0b100: 优化方案1 随机选取pivot
            * 0b010: 优化方案2 减少交换次数
            * 0b001: 优化方案3 当待排序区间较小时，改用插入排序
    返回值：
        bool类型：是否排序成功
    '''
    if not all([str(i).isdigit() for i in data]):
        return False
    
    lenData = len(data)
    
    quickSort(data, 0, lenData-1, optimize)
    
    if reverse:
        data = data[::-1]
    return True


def quickSort(data, l, r, opt=0b111):
    '''实现快速排序'''
    if l >= r: return
    
    if opt & 0b001 and r - l < 10: # 优化方案3 当待排序区间较小时，改用插入排序
        insert_sort(data[l:r+1])
        return
    
    # 选取中心轴pivot
    if opt & 0b100: # 优化方案1 随机选取pivot
        randInt = random.randint(l, r)
        swap(data, l, randInt)
    pivot = data[l]

    # 以pivot为中轴将数据分成小堆和大堆
    i, j = l, r
    while i < j:
        if opt & 0b010: # 优化方案2 减少交换次数
            while data[j] >= pivot and i < j: j -= 1
            while data[i] <= pivot and i < j: i += 1
            if i<j: swap(data, i, j)
            else: swap(data, i, l) 
        else:
            while data[j] >= pivot and i < j: j -= 1
            swap(data, i, j) 

            while data[i] <= pivot and i < j: i += 1
            swap(data, i, j) 
            
        
    # 对Pivot左边和右边的数据进行同样的操作
    m = i
    quickSort(data, l, m-1, opt)
    quickSort(data, m+1, r, opt)

In [14]:
import copy

data = copy.copy(repeatData)
res = quick_sort(data)
# res, data, repeatData
res

True

## 归并排序

## 堆排

## 桶排序

# 测试各算法的效率 （用多进程分别实现）

In [26]:
from multiprocessing import Pool,Lock
from itertools import product
import time
import copy

dataDict = {
    'random data': copy.copy(randomData),
    'reversed data': copy.copy(reversedData),
    'almost sorted data': copy.copy(almostSortedData),
    'sorted data': copy.copy(sortedData),
    'repeat data': copy.copy(repeatData)
}

sort_funcs = (select_sort,
                insert_sort,
                bubble_sort,
                quick_sort)

def worker(args):
    data, func = args
    
    st = time.time()
    res = func(dataDict[data])
    ed = time.time()
    
    t = (ed-st)
    print(data.ljust(18),func.__name__.ljust(11), f"执行完毕，用时{t:.8f}秒,", "返回结果：", res)
    return res,t
    
it = product(dataDict.keys(), sort_funcs)

# with Pool() as pool:
#     pool.map(func=worker, iterable=it)

for args in it:
    worker(args)
    
# pool.join()
print("end")

random data        select_sort 执行完毕，用时0.11385512秒, 返回结果： True
random data        insert_sort 执行完毕，用时0.13192225秒, 返回结果： True
random data        bubble_sort 执行完毕，用时0.13472581秒, 返回结果： True
random data        quick_sort  执行完毕，用时0.00324321秒, 返回结果： True
reversed data      select_sort 执行完毕，用时0.11048722秒, 返回结果： True
reversed data      insert_sort 执行完毕，用时0.13608527秒, 返回结果： True
reversed data      bubble_sort 执行完毕，用时0.13728976秒, 返回结果： True
reversed data      quick_sort  执行完毕，用时0.00318289秒, 返回结果： True
almost sorted data select_sort 执行完毕，用时0.10979509秒, 返回结果： True
almost sorted data insert_sort 执行完毕，用时0.13884473秒, 返回结果： True
almost sorted data bubble_sort 执行完毕，用时0.14141965秒, 返回结果： True
almost sorted data quick_sort  执行完毕，用时0.00332212秒, 返回结果： True
sorted data        select_sort 执行完毕，用时0.11127543秒, 返回结果： True
sorted data        insert_sort 执行完毕，用时0.14097261秒, 返回结果： True
sorted data        bubble_sort 执行完毕，用时0.13779449秒, 返回结果： True
sorted data        quick_sort  执行完毕，用时0.00339437秒, 返回结果： True
repeat d

In [ ]:
dataDict = {
    'random data': copy.copy(randomData),
    'reversed data': copy.copy(reversedData),
    'almost sorted data': copy.copy(almostSortedData),
    'sorted data': copy.copy(sortedData),
    'repeat data': copy.copy(repeatData)
}
for data in dataDict.values():
    print(data)